4. **_Hypothesis_**:  
Some users are more likely to badly rate a book, especially the unknowns
   - **Metric**: Rating score
  
- **Missing Values**:

  - `profileName`: set missing values as unknown
  - `review/score`: remove the entire sample

- **Data Transformation**:

  ***

In [1]:
#Import libraries

import pymongo
import pandas as pd

In [2]:
# Connect to MongoDB

client = pymongo.MongoClient('mongodb://localhost:27017/')
database = client['spark_db']
books = database['book_reviews']

In [4]:
# Define the aggregation pipeline
pipeline = [
    {
        "$project": {
            "profileName": {
                "$ifNull": ["$profileName", "Unknown"]
            },
            "review/score": 1
        }
    }
]

# Execute the aggregation query and create a DataFrame
df = pd.DataFrame(list(books.aggregate(pipeline)))

# Drop the '_id' column from the DataFrame
df = df.drop('_id', axis=1)

# Calculate the mean review score for each 'profileName' and sort the result by review score in descending order
result = df.groupby('profileName').mean().sort_values(
    by='review/score', ascending=False)

# Count the number of reviews for each 'profileName' and add it to the result DataFrame
result['N_reviews'] = df['profileName'].value_counts()

# Sort the result by the number of reviews in descending order
result = result.sort_values(by='N_reviews', ascending=False)

result

,review/score,N_reviews
profileName,,
Unknown,4.161831,56658
Midwest Book Review,4.996503,572
A Customer,4.119760,501
Harriet Klausner,4.626703,367
E. A Solinas ea_solinas,4.762215,307
...,...,...
T. Kessler Tom Drama,5.000000,1
T. Jury,5.000000,1
T. Keune Bookman,5.000000,1
